In [1]:
import numpy as np
import tensorflow as tf
import os
import cv2
from sklearn.model_selection import train_test_split

In [2]:
img_folder = 'archive/lfw-deepfunneled/lfw-deepfunneled'

X = []
Y = []
labels = []

# Create haar cascade model
haar_cascade = cv2.CascadeClassifier('haar_face.xml')

In [3]:
def restrained_cpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
def createLabels():
    for subdir in os.listdir(img_folder):
        subdir_path = os.path.join(img_folder, subdir)

        # Only process directories
        if os.path.isdir(subdir_path):
            labels.append(subdir)

In [5]:
def importImages_Labels():
    for label in labels:
        path = os.path.join(img_folder, label)
        # Loop through each image in the sub-folder
        for image_name in os.listdir(path):
            image_path = os.path.join(path, image_name)

            # Load the image using OpenCV
            image = cv2.imread(image_path)

            if image is not None:
                # # Convert image to grayscale
                # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                #
                # # detect face in the image (increase image size 10% and minimal neighbors = 3)
                # faces_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)
                #
                # for (x, y, w, h) in faces_rect:
                #     faces_roi = gray[y:y + h, x:x + w]
                #     X.append(faces_roi)
                #     Y.append(labels.index(label))
                X.append(image)
                Y.append(labels.index(label))

            else:
                print(f"Warning: Could not load image {image_path}")

    print("Importing images and labels completed!")

In [6]:
def train_model(X_train, Y_train):
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(X_train, Y_train)
    print("Model training completed!")
    return model

In [7]:
def test_model(model, X_test, Y_test):
    # Initialize lists to hold the results
    predictions = []
    correct_labels = []
    loop = len(X_test)
    count = 0
    # Loop through each of the first 500 images
    for i in range(loop):
        test_image = X_test[i]
        
        gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
        
        faces_rect = haar_cascade.detectMultiScale(gray, 1.3, 4)
        
        for (x, y, w, h) in faces_rect:
            faces_roi = gray[y:y + h, x:x + w]
            label, confidence = model.predict(faces_roi)
        
        if labels[label] == labels[Y_test[i]]:
            count += 1
            
        # Print the prediction and actual label
        correct_percent = count / loop * 100
        print(f'{i} / {loop} ({correct_percent}) Predicted Label: {labels[label]}, Confidence: {confidence}, Actual Label: {labels[Y_test[i]]}')

        # Check if the prediction is correct
        # if labels[label] == labels[Y_test[i]]:
        #     img = X_test[i]  # The original image in color or grayscale
        # 
        #     # Annotate the image with the predicted label
        #     cv2.putText(gray, str(labels[label]), (20, 20), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), thickness=2)
        #     cv2.rectangle(gray, (0, 0), (250, 250), (0, 255, 0), 2)
        #     cv2.imshow('Detected Face', gray)
        # 
        #     cv2.waitKey(0)
        #     cv2.destroyAllWindows()

        # Append the predicted label and the true label to the lists
        predictions.append(label)
        correct_labels.append(Y_test[i])

    # Calculate accuracy
    correct_predictions = sum([1 for p, c in zip(predictions, correct_labels) if p == c])
    accuracy = correct_predictions / len(correct_labels) * 100

    # Print results
    print(f"Total test images: {len(correct_labels)}")
    print(f"Correct predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.2f}%")

In [8]:
restrained_cpu()

In [9]:
createLabels()
importImages_Labels()

Importing images and labels completed!


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [11]:
X_train_gray_rect = []
Y_train_gray_rect = []

for idx, img in enumerate(X_train):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_rect = haar_cascade.detectMultiScale(gray, 1.3, 4)
    for (x, y, w, h) in faces_rect:
        faces_roi = gray[y:y + h, x:x + w]
        X_train_gray_rect.append(faces_roi)
        Y_train_gray_rect.append(Y_train[idx])
        

In [12]:
X_train_np = np.array(X_train_gray_rect)
Y_train_np = np.array(Y_train_gray_rect)

trained_model = train_model(X_train_np, Y_train_np)

C:\Users\antho\AppData\Local\Temp\ipykernel_14848\2215841435.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_np = np.array(X_train_gray_rect)


Model training completed!


In [13]:
test_model(trained_model, X_test, Y_test)

0 / 3309 (0) Predicted Label: Veronica_Lake, Confidence: 75.5032013919274, Actual Label: Tim_Conway
1 / 3309 (1) Predicted Label: Mahmoud_Abbas, Confidence: 80.24200970348151, Actual Label: Mahmoud_Abbas
2 / 3309 (1) Predicted Label: Kristanna_Loken, Confidence: 83.15119171353314, Actual Label: Michelangelo_Antonioni
3 / 3309 (1) Predicted Label: Jean_Chretien, Confidence: 70.56178624279111, Actual Label: Abraham_Foxman
4 / 3309 (1) Predicted Label: Julianne_Moore, Confidence: 76.7283427650586, Actual Label: Mona_Rishmawi
5 / 3309 (1) Predicted Label: Gerhard_Schroeder, Confidence: 74.5184850616343, Actual Label: Barbara_Brezigar
6 / 3309 (2) Predicted Label: John_Ashcroft, Confidence: 74.0945737291841, Actual Label: John_Ashcroft
7 / 3309 (2) Predicted Label: Ray_Romano, Confidence: 83.11768316863439, Actual Label: Michael_Andretti
8 / 3309 (2) Predicted Label: Fred_Thompson, Confidence: 85.07425135408154, Actual Label: Tony_Parker
9 / 3309 (2) Predicted Label: JP_Suarez, Confidence: 